In [ ]:
import django_initialiser
from api.melody_utils.search import search_lcs, search_dtw
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
input_search = [76, 75, 76, 75, 76, 71, 74, 72, 69, 57, 76, 75, 76, 75, 76, 71, 74, 72, 69, 57, 76, 75, 76, 75, 76, 71, 74, 72, 69, 57, 76, 75, 76, 75, 76, 71, 74, 72, 69, 57]

In [ ]:
try:
    f = open('./results.csv', 'a')
    for i in range (22, 41):
        cur_search = input_search[:i]
        lcs = search_lcs(cur_search, return_time=True)
        fast_dtw = search_dtw(cur_search, 0, generate_graphs=False, fast=True, return_time=True)
        dtw = search_dtw(cur_search, 0, generate_graphs=False, fast=False, return_time=True)
        f.write(f'{i},{lcs},{fast_dtw},{dtw}\n')
        print(f'Len {i} done')
    f.close()

except KeyboardInterrupt:
    f.close()
    pass

In [ ]:
df = pd.read_csv('results.csv')
print(df)
# plt.plot(pd['lcs_time'])
plt.plot(df['input_len'], df['lcs_time'], label='LCS')
plt.plot(df['input_len'], df['fast_dtw_time'], label='fastDTW')
plt.plot(df['input_len'], df['dtw_time'], label='DTW')
plt.ylabel("Search time (s)")
plt.xlabel("Query melody length")
plt.legend()
plt.savefig('./query_len.png')
plt.show()

In [ ]:
constant = 0

for i in range(0, 38):
    constant += df['dtw_time'][i] / df['lcs_time'][i]

constant = constant / len(df['lcs_time'])
print(constant)